# Spark assignment

Taratukhin Oleg, C41323

Initial env setup:

In [20]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pathlib import Path
import emoji

spark = SparkSession.builder \
    .appName("itmo spark assignment") \
    .master("local[*]") \
    .getOrCreate()

spark

In [35]:
!cd data && ls
!rm -rf results/task5/

PATH_TO_DATA = Path("data/")
PATH_TO_RESULTS = Path("results/task5/")

followers.parquet              posts_api.json
followers_posts_api_final.json posts_likes.parquet
followers_posts_likes.parquet


Utilities:

In [22]:
def save_as_json(df: pyspark.sql.dataframe.DataFrame, name: str):
    df.write \
        .format("json") \
        .save(str(PATH_TO_RESULTS / name))

In [30]:
followers_posts_likes = spark \
    .read \
    .parquet(str(PATH_TO_DATA / "followers_posts_likes.parquet"))

followers_posts_likes.limit(10).toPandas()

,itemType,ownerId,itemId,likerId
0,post,1567,3324,2767
1,post,1567,3324,254280
2,post,1567,3324,553430
3,post,1567,3324,905379
4,post,1567,3324,2411502
5,post,1567,3324,5274303
6,post,1567,3324,50038414
7,post,1567,3324,84207602
8,post,1567,3324,145422426
9,post,1567,3331,4601081


## Task 5

 Find friends. The idea is, if users like each other posts, then they are friends.

In [36]:
friends = followers_posts_likes.alias("df1") \
    .join(followers_posts_likes.alias("df2"),
          ((col("df1.ownerId") == col("df2.likerId")) &
           (col("df1.ownerId") == col("df2.likerId"))), 'leftsemi') \
    .distinct() \
    .orderBy("df1.ownerId") \
    .select("ownerId", "likerId")

save_as_json(friends, "task5_friends.json")

friends.limit(10).toPandas()

,ownerId,likerId
0,637,87335
1,637,23304482
2,637,3888416
3,637,41311426
4,637,50531713
5,637,422527
6,637,357278
7,637,1998076
8,637,1401589
9,637,425615
